In [ ]:
from math import pi
import numpy
%matplotlib inline
from matplotlib import pyplot
from matplotlib import rcParams
rcParams['font.family']='serif'
rcParams['font.size']=16
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
import numba
from numba import jit

In [ ]:
def plot_3D(x,y,u):    
    fig = pyplot.figure(figsize=(11,7), dpi=100)
    ax = fig.gca(projection='3d')
    X,Y = numpy.meshgrid(x,y)
    surf = ax.plot_surface(X,Y,u[:], rstride=1, cstride=1, cmap=cm.viridis,
            linewidth=0, antialiased=False)

    ax.set_xlim(0,1)
    ax.set_ylim(0,1)
    ax.set_xlabel('$x$')
    ax.set_ylabel('$y$')
    ax.set_zlabel('$u$')
    ax.view_init(30,45)

In [ ]:
def poisson_initial(ni,nj):
    u0=numpy.random.random((nj+1,ni+1))
    dx=1./(ni-1)
    dy=1./(nj-1)
    x=numpy.linspace(0-dx/2,1+dx/2,ni+1)
    y=numpy.linspace(0-dy/2,1+dy/2,nj+1)
    plot_3D(x,y,u0)
    return u0

In [ ]:
u0=poisson_initial(41,41)

In [ ]:
@jit
def jacobi(ni,nj,n,res_target):
    dx=1./(ni-1)
    dy=1./(nj-1)
    x=numpy.linspace(0-dx/2,1+dx/2,ni+1)
    y=numpy.linspace(0-dy/2,1+dy/2,nj+1)
    
    X,Y=numpy.meshgrid(x,y)
    f=-2*(2*pi*n)**2*numpy.sin(2*pi*n*X)*numpy.sin(2*pi*n*Y)
    u_ex=numpy.sin(2*pi*n*X)*numpy.sin(2*pi*n*Y)
    u0=numpy.random.random((nj+1,ni+1))
    u=u0.copy()
    u[0,:]=-u[1,:]
    u[-1,:]=-u[-2,:]
    u[1:-1,0]=-u[1:-1,1]
    u[1:-1,-1]=-u[1:-1,-2]
    iterations=0
    dif=numpy.zeros((nj+1,ni+1))
    err=numpy.zeros((nj+1,ni+1))
    res=numpy.zeros((nj+1,ni+1))
    res_max=1e3
    
    while res_max>res_target:
        uold=u.copy()
        u[1:-1,1:-1]=1/(2*(dx**2+dy**2))*((uold[1:-1,2:]+uold[1:-1,:-2])*dy**2+\
                     (uold[2:,1:-1]+uold[:-2,1:-1])*dx**2-f[1:-1,1:-1]*dx**2*dy**2)
        
        #for j in range(1,nj-1):
            #for i in range(1,ni-1):
            if j=1
            
            if i==1
            
                #u[j,i]=1./(2*(dx**2+dy**2))*((uold[j,i+1]+uold[j,i-1])*dy**2+\
                       #(uold[j+1,i]+uold[j-1,i])*dx**2-f[j,i]*dx**2*dy**2)
                    
                    
        u[0,:]=-u[1,:]
        u[-1,:]=-u[-2,:]
        u[1:-1,0]=-u[1:-1,1]
        u[1:-1,-1]=-u[1:-1,-2]
        
        for j in range(1,nj):
            for i in range(1,ni):
                res[j,i]=numpy.abs(f[j,i]-(u[j,i-1]-2*u[j,i]+u[j,i+1])/dx**2-\
                                  (u[j-1,i]-2*u[j,i]+u[j+1,i])/dy**2)
        res_max=numpy.max(res)
                   
        iterations+=1
        
    for j in range(nj+1):
        for i in range(ni+1):
            dif[j,i]=numpy.abs(u[j,i]-uold[j,i])
            err[j,i]=numpy.abs(u[j,i]-u_ex[j,i])
    
    dif_max=numpy.max(dif)
    err_max=numpy.max(err)
        
    A=numpy.array([u,x,y,dif_max,err_max,res_max,iterations])
    
    return A

In [ ]:
@jit
def SOR(ni,nj,n,res_target,omega):
    dx=1./(ni-1)
    dy=1./(nj-1)
    x=numpy.linspace(0-dx/2,1+dx/2,ni+1)
    y=numpy.linspace(0-dy/2,1+dy/2,nj+1)
    
    X,Y=numpy.meshgrid(x,y)
    f=-2*(2*pi*n)**2*numpy.sin(2*pi*n*X)*numpy.sin(2*pi*n*Y)
    u_ex=numpy.sin(2*pi*n*X)*numpy.sin(2*pi*n*Y)
    u0=numpy.random.random((nj+1,ni+1))
    u=u0.copy()
    u[0,:]=u_ex[0,:]
    u[-1,:]=u_ex[-1,:]
    u[:,0]=u_ex[:,0]
    u[:,-1]=u_ex[:,-1]
    dif=numpy.zeros((nj+1,ni+1))
    err=numpy.zeros((nj+1,ni+1))
    res=numpy.zeros((nj+1,ni+1))
    res_max=1e3
    iterations=0
    
    while res_max>res_target:
        uold=u.copy()
        for j in range(1,nj):
            for i in range(1,ni):
                u[j,i]=omega/(2*(dx**2+dy**2))*((u[j,i-1]+u[j,i+1])*dy**2+\
                       (u[j-1,i]+u[j+1,i])*dx**2-f[j,i]*dx**2*dy**2)+\
                       (1-omega)*u[j,i]
        
        u[0,:]=u_ex[0,:]
        u[-1,:]=u_ex[-1,:]
        u[:,0]=u_ex[:,0]
        u[:,-1]=u_ex[:,-1]
        
        for j in range(1,nj):
            for i in range(1,ni):
                res[j,i]=numpy.abs(f[j,i]-(u[j,i-1]-2*u[j,i]+u[j,i+1])/dx**2-\
                                  (u[j-1,i]-2*u[j,i]+u[j+1,i])/dy**2)
        res_max=numpy.max(res)
        
        iterations+=1
        
    for j in range(nj+1):
        for i in range(ni+1):
            dif[j,i]=numpy.abs(u[j,i]-uold[j,i])
            err[j,i]=numpy.abs(u[j,i]-u_ex[j,i])
            
    dif_max=numpy.max(dif)
    err_max=numpy.max(err)
    
    A=numpy.array([u,x,y,dif_max,err_max,res_max,iterations])
    
    return A

In [ ]:
A40_jac1=jacobi(41,41,1,1e-9)
plot_3D(A40_jac1[1],A40_jac1[2],A40_jac1[0])

In [ ]:
A40_SOR1=SOR(41,41,1,1e-3,1.9)
plot_3D(A40_SOR1[1],A40_SOR1[2],A40_SOR1[0])

In [ ]:
error40_jac=[]
for k in range(1,20):
    error40_jac.append(jacobi(41,41,k,1e-9)[4])
    
pyplot.figure(figsize=(10,6))
pyplot.grid(True)
pyplot.xlabel('$n$')
pyplot.ylabel('error')
pyplot.plot(numpy.linspace(1,19,19),error40_jac,'o')
pyplot.title('Error for different n values, ni=nj=41',fontsize=18)
pyplot.legend()

In [ ]:
error_grid_jac=[]
ninj=[]
for ni in range(40,81,20):
    for nj in range(40,81,20):
        error_grid_jac.append(jacobi(ni,nj,1,1e-9)[4])
        ninj.append(ni*nj)
        
pyplot.figure(figsize=(10,6))
pyplot.grid(True)
pyplot.xlabel('$ni \cdot nj$')
pyplot.ylabel('error')
pyplot.plot(ninj,error_grid_jac,'o')
pyplot.title('Error for different grid numbers, n=1',fontsize=18)
pyplot.legend()

In [ ]:
error40_SOR=[]
for k in range(1,20):
    error40_SOR.append(SOR(41,41,k,1e-9,1.9)[4])
    
pyplot.figure(figsize=(10,6))
pyplot.grid(True)
pyplot.xlabel('$n$')
pyplot.ylabel('error')
pyplot.plot(numpy.linspace(1,19,19),error40_SOR,'o')
pyplot.title('Error for different n values, ni=nj=41',fontsize=18)
pyplot.legend()

In [ ]:
error_grid_SOR=[]
ninj=[]
for ni in range(40,81,20):
    for nj in range(40,81,20):
        error_grid_SOR.append(SOR(ni,nj,1,1e-9,1.9)[4])
        ninj.append(ni*nj)
        
pyplot.figure(figsize=(10,6))
pyplot.grid(True)
pyplot.xlabel('$ni \cdot nj$')
pyplot.ylabel('error')
pyplot.plot(ninj,error_grid_SOR,'o')
pyplot.title('Error for different grid numbers, n=1',fontsize=18)
pyplot.legend()

In [ ]:
error40_SOR_omega=[]
k=1.7
while k<1.96:
    error40_SOR_omega.append(SOR(41,41,1,1e-9,k)[6])
    k+=0.01
pyplot.figure(figsize=(10,6))
pyplot.grid(True)
pyplot.xlabel('$\omega$')
pyplot.ylabel('number of iterations')
pyplot.plot(numpy.linspace(1.7,1.95,(1.95-1.7)/0.01+1),error40_SOR_omega,'o')
pyplot.title('Error for different $\omega$ values, n=1, ni=nj=41',fontsize=18)
pyplot.legend()

In [ ]:
u40_jac1,x40_jac1,y40_jac1,dif40_jac1,err40_jac1,res40_jac1,iterations40_jac1=jacobi(41,41,1,1e-9)
plot_3D(x40_jac1,y40_jac1,u40_jac1)
print('dif40_jacobi1={}'.format(dif40_jac1))
print('err40_jacobi1={}'.format(err40_jac1))
print('res40_jacobi1={}'.format(res40_jac1))
print('iterations40_jacobi1={}'.format(iterations40_jac1))

In [ ]:
u_jac2,x_jac2,y_jac2,dif_jac2,err_jac2,res_jac2,iterations_jac2=jacobi(41,41,2,1e-9)
plot_3D(x_jac2,y_jac2,u_jac2)
print('dif_jacobi2={}'.format(dif_jac2))
print('err_jacobi2={}'.format(err_jac2))
print('res_jacobi2={}'.format(res_jac2))
print('iterations_jacobi2={}'.format(iterations_jac2))

In [ ]:
A40_jac3=jacobi(41,41,3,1e-9)
print('dif_jacobi3={}'.format(A40_jac3[3]))
print('err_jacobi3={}'.format(A40_jac3[4]))
print('res_jacobi3={}'.format(A40_jac3[5]))
print('iterations_jacobi3={}'.format(A40_jac3[6]))

In [ ]:
u_jac4,x_jac4,y_jac4,dif_jac4,err_jac4,res_jac4,iterations_jac4=jacobi(41,41,4,1e-9)
print('dif_jacobi4={}'.format(dif_jac4))
print('err_jacobi4={}'.format(err_jac4))
print('res_jacobi4={}'.format(res_jac4))
print('iterations_jacobi4={}'.format(iterations_jac4))

In [ ]:
u80_jac1,x80_jac1,y80_jac1,dif80_jac1,err80_jac1,res80_jac1,iterations80_jac1=jacobi(81,81,1,1e-9)
plot_3D(x80_jac1,y80_jac1,u80_jac1)
print('dif80_jacobi1={}'.format(dif80_jac1))
print('err80_jacobi1={}'.format(err80_jac1))
print('res80_jacobi1={}'.format(res80_jac1))
print('iterations80_jacobi1={}'.format(iterations80_jac1))

In [ ]:
A40_SOR1=SOR(41,41,1,1e-9,1.9)
plot_3D(A40_SOR1[1],A40_SOR1[2],A40_SOR1[0])
print('dif_SOR1={}'.format(A40_SOR1[3]))
print('err_SOR1={}'.format(A40_SOR1[4]))
print('res_SOR1={}'.format(A40_SOR1[5]))
print('iterations_SOR1={}'.format(A40_SOR1[6]))

In [ ]:
u_SOR2,x_SOR2,y_SOR2,dif_SOR2,err_SOR2,res_SOR2,iterations_SOR2=SOR(41,41,2,1e-9,1.9)
plot_3D(x_SOR2,y_SOR2,u_SOR2)
print('dif_SOR2={}'.format(dif_SOR2))
print('err_SOR2={}'.format(err_SOR2))
print('res_SOR2={}'.format(res_SOR2))
print('iterations_SOR2={}'.format(iterations_SOR2))

In [ ]:
u_SOR3,x_SOR3,y_SOR3,dif_SOR3,err_SOR3,res_SOR3,iterations_SOR3=SOR(41,41,3,1e-9,1.9)
plot_3D(x_SOR3,y_SOR3,u_SOR3)
print('dif_SOR3={}'.format(dif_SOR3))
print('err_SOR3={}'.format(err_SOR3))
print('res_SOR3={}'.format(res_SOR3))
print('iterations_SOR3={}'.format(iterations_SOR3))